## Build PPI Network (Yashwi)

In [1]:
import numpy as np
from collections import defaultdict
import scipy.stats as stat
import pandas as pd
import time, os, random
import networkx as nx
from networkx.algorithms.link_analysis import pagerank

In [3]:
exec(open('D:/IUB/Classes/ML for Bioinformatics/Project/utilities/network_utilities_ver4.py').read())
exec(open('D:/IUB/Classes/ML for Bioinformatics/Project/utilities/useful_utilities.py').read())
start = time.ctime()

In [5]:
## Initialize
string_cutoff = 700
fo_dir = 'D:/IUB/Classes/ML for Bioinformatics/Project/result'

In [7]:
## Prepare data

#  Load immunotherapy biomarkers
ib = pd.read_csv('D:/IUB/Classes/ML for Bioinformatics/Project/data/Marker_summary.txt', sep='\t')

In [9]:
#  Construct Network (STRING network v11)
print('constructing STRING PPI network, %s'%time.ctime())
tmp_G = nx.Graph()
annotation = pd.read_csv('D:/IUB/Classes/ML for Bioinformatics/Project/data/9606.protein.aliases.v11.0.txt.gz', sep="\t", header=None, comment="#", names=["string_protein_id", "alias", "source"], usecols=[0, 1, 2]) # STRING network ensembl-geneID mapping
net = pd.read_csv('D:/IUB/Classes/ML for Bioinformatics/Project/data/9606.protein.links.v11.0.txt.gz', sep=' ') # STRING network
nodes1 = net.values[:,0]
nodes2 = net.values[:,1]
scores = net.values[:,2]
for n1, n2, score in zip(nodes1, nodes2, scores):
        if score >= string_cutoff:
                tmp_G.add_edge(n1, n2)
LCC_genes = max(nx.connected_components(tmp_G), key=len)
G = tmp_G.subgraph(LCC_genes) ## Largest Connected Componenets
network_nodes = G.nodes()
network_edges = G.edges()
print('network nodes: %s'%len(network_nodes))
print('network edges: %s'%len(network_edges))
print('\n')

print("Columns in annotation file:", annotation.columns)
#  annotation
print('make annotation dictionary, ', time.ctime())
anno = pd.DataFrame(data=annotation.loc[annotation['source'].str.contains('HUGO', na=False),:])
anno_dic = defaultdict(list) # { ensp : [ genes ] }
for ensp, gene in zip(anno['string_protein_id'].tolist(), anno['alias'].tolist()):
        anno_dic[ensp].append(gene)

print("Columns in annotation file:", annotation.columns)

constructing STRING PPI network, Sat May  3 16:37:53 2025
network nodes: 16957
network edges: 420381


Columns in annotation file: Index(['string_protein_id', 'alias', 'source'], dtype='object')
make annotation dictionary,  Sat May  3 16:38:11 2025
Columns in annotation file: Index(['string_protein_id', 'alias', 'source'], dtype='object')


In [11]:
## Network propagation
print('run network propagation, %s'%time.ctime())

for biomarker, feature in zip(ib['Name'].tolist(), ib['Feature'].tolist()):
        if '%s.txt'%biomarker in os.listdir(fo_dir):
                continue
        if not 'target' in feature:
                continue
        print('\ttesting %s, %s'%(biomarker, time.ctime()))

        output = defaultdict(list)
        output_col = ['gene_id', 'string_protein_id', 'propagate_score']

        # network propagation
        biomarker_genes = ib.loc[ib['Name']==biomarker,:]['Gene_list'].tolist()[0].split(':')
        pIDs = annotation.loc[annotation['alias'].isin(biomarker_genes),:]['string_protein_id'].tolist() #only remain biomarker_gene
        propagate_input = {}
        for node in network_nodes:
                propagate_input[node] = 0
                if node in pIDs:
                        propagate_input[node] = 1
        propagate_scores = pagerank(G, personalization=propagate_input, max_iter=100, tol=1e-06) ## NETWORK PROPAGATION

        # output
        for ensp in list(propagate_scores.keys()):
                geneID = 'NA'
                if ensp in list(anno_dic.keys()):
                        for gene in anno_dic[ensp]:
                                geneID = gene
                                output['gene_id'].append(geneID)
                                output['string_protein_id'].append(ensp)
                                output['propagate_score'].append(propagate_scores[ensp])
                else:
                        geneID = 'NA'
                        output['gene_id'].append(geneID)
                        output['string_protein_id'].append(ensp)
                        output['propagate_score'].append(propagate_scores[ensp])
        output = pd.DataFrame(data=output, columns=output_col)
        output.to_csv('%s/%s.txt'%(fo_dir, biomarker), sep='\t', index=False)

end = time.ctime()
print('process complete // start: %s, end: %s'%(start, end))

run network propagation, Sat May  3 16:38:40 2025
process complete // start: Sat May  3 16:37:42 2025, end: Sat May  3 16:38:40 2025


## Build Network Graph as input to Models
Don't run again

In [17]:
import pandas as pd
import torch
from torch_geometric.data import Data

In [19]:
# 1) Filter STRING edges
edges_df = (
    net[['protein1','protein2','combined_score']]
    .query("combined_score >= @string_cutoff")
    .loc[:, ['protein1','protein2']]
    .drop_duplicates()
    .reset_index(drop=True)
)

In [21]:
# 2) Build node2idx mapping
nodes = pd.unique(edges_df[['protein1','protein2']].values.ravel())
nodes = list(nodes)
node2idx = {node: i for i, node in enumerate(nodes)}

In [23]:
# 3) Build undirected edge_index tensor
src = edges_df['protein1'].map(node2idx).tolist()
dst = edges_df['protein2'].map(node2idx).tolist()
edge_index = torch.tensor([src + dst, dst + src], dtype=torch.long)

In [ ]:
import pandas as pd

# Load the clinical response file
clinical = pd.read_csv('D:/IUB/Classes/ML for Bioinformatics/Project/data/Kim_et_al/clinical.txt', sep='\t')

# Extract just the ID → response mapping and save
labels_df = clinical[['ID', 'response']].set_index('ID')
labels_df.to_csv('response_labels.csv')

In [ ]:
import pandas as pd

# Load the clinical response file
clinical = pd.read_csv('D:/IUB/Classes/ML for Bioinformatics/Project/data/Kim_et_al/expression_mRNA.norm3.txt', sep='\t')

# Extract just the ID → response mapping and save
#labels_df = clinical[['ID', 'response']].set_index('ID')
clinical.to_csv('expression_labels.csv')

In [35]:
import pandas as pd

# 1) Peek at your file’s first few rows & columns
expr_raw = pd.read_csv('D:/IUB/Classes/ML for Bioinformatics/Project/data/Kim_et_al/expression_labels.csv', sep=',')  # or sep='\t' if it’s tab-delim
print("Columns:", expr_raw.columns[:10].tolist())
print(expr_raw.head())

# 2) If you see metadata columns (e.g. Length, Chr, Start, End, Strand), drop them:
meta_cols = [c for c in expr_raw.columns 
             if c.lower() in ('length','chr','start','end','strand')]
expr = expr_raw.drop(columns=meta_cols, errors='ignore')

# 3) Make the first column your index (gene IDs) if it isn’t already:
#    (replace 'Geneid' below with whatever that header actually is)
if expr.columns[0].lower() not in ('ensp','gene','geneid'):
    expr = expr.set_index(expr.columns[0])
else:
    expr = expr.set_index(expr.columns[0])

# 4) If by chance genes are columns and samples are rows, transpose:
#    after the last step genes should be the index and samples the columns.
if expr.shape[0] < expr.shape[1]:
    expr = expr.T

# # 5) Now you have expr: rows=genes, cols=samples.  Save your clean matrix:
# expr.to_csv('expression_matrix.csv')
# print("Cleaned expression matrix shape:", expr.shape)

Columns: ['Unnamed: 0', 'gene_id', 'PB-16-002', 'PB-16-003', 'PB-16-004', 'PB-16-005', 'PB-16-018', 'PB-16-019', 'PB-16-020', 'PB-16-021']
   Unnamed: 0  gene_id  PB-16-002  PB-16-003  PB-16-004  PB-16-005  PB-16-018  \
0           0     A1BG   0.275921  -2.019086  -0.938814  -0.391196   1.027154   
1           1     A1CF  -1.124338  -0.391196   0.857254  -0.640667   1.512390   
2           2      A2M  -0.857254   0.450744   0.781034  -0.391196  -2.019086   
3           3    A2ML1   0.164211  -0.938814  -0.575109  -0.857254   0.000000   
4           4  A3GALT2   1.233495  -0.543252   0.511936  -0.543252   0.219723   

   PB-16-019  PB-16-020  PB-16-021  ...  PB-16-057  PB-16-059  PB-16-060  \
0   1.359737  -0.781034  -0.164211  ...  -0.450744   1.711675   0.219723   
1   0.575109  -1.233495   0.000000  ...  -0.109200   0.109200   0.333005   
2  -1.027154  -1.359737  -0.938814  ...   0.709103   0.000000  -0.054519   
3   0.709103   0.938814  -1.124338  ...   1.027154  -0.709103   0.3911

In [31]:
# 4) Load your expression & labels (make sure paths are correct)
expr_df  = pd.read_csv('D:/IUB/Classes/ML for Bioinformatics/Project/data/Kim_et_al/expression_matrix.csv', index_col=0)
labels_df = pd.read_csv('D:/IUB/Classes/ML for Bioinformatics/Project/data/Kim_et_al/response_labels.csv', index_col=0)

In [37]:
# 5) Align expression rows to our node order, fill missing genes with zero
expr_df = (
    expr_df
      .reindex(nodes)
      .fillna(0)
      .astype(float)        # or .astype('int64'), etc.
)
expr_df = expr_df.reindex(nodes).fillna(0)

In [39]:
import torch
from torch_geometric.data import Data

# 1) Find samples present in both expression and labels
common_samples = expr_df.columns.intersection(labels_df.index)

# 2) Warn about any mismatches
extra = set(expr_df.columns) - set(common_samples)
if extra:
    print(f"Dropping non-sample columns from expr_df: {extra}")
missing = set(labels_df.index) - set(common_samples)
if missing:
    print(f"No expression data for these labels, dropping them: {missing}")

# 3) Reindex expr_df to only those common samples (optional, for cleanliness)
expr_df = expr_df[common_samples]

# 4) Build Data objects
data_list = []
for sample in common_samples:
    x = torch.tensor(expr_df[sample].values, dtype=torch.float).unsqueeze(1)  # [num_nodes × 1]
    y = torch.tensor([labels_df.loc[sample, 'response']], dtype=torch.long)    # [1]
    data_list.append(Data(x=x, edge_index=edge_index, y=y))

print(f"Built {len(data_list)} graph examples, each with {expr_df.shape[0]} nodes.")

Dropping non-sample columns from expr_df: {'gene_id'}
Built 45 graph examples, each with 17185 nodes.


In [41]:
# 6) Build a Data object per sample
from torch_geometric.data import Data
data_list = []
for sample in expr_df.columns:
    x = torch.tensor(expr_df[sample].values, dtype=torch.float).unsqueeze(1)  # [num_nodes × 1]
    y = torch.tensor([labels_df.loc[sample, 'response']], dtype=torch.long)    # [1]
    data_list.append(Data(x=x, edge_index=edge_index, y=y))

print(f"Built {len(data_list)} graphs, each with {len(nodes)} nodes.")

Built 45 graphs, each with 17185 nodes.


## GNN Pipeline for Gastric Cancer Cohort (Kim et al.) and TCGA Projection

In [43]:
# 1) Load the full STRING PPI table (make sure you’ve downloaded and un-zipped it)
net = pd.read_csv('D:/IUB/Classes/ML for Bioinformatics/Project/data/9606.protein.links.v11.0.txt.gz', sep=' ',
    usecols=['protein1','protein2','combined_score'])

# 2) Filter by your confidence cutoff
filtered = (
    net[net.combined_score >= string_cutoff]
    .loc[:, ['protein1','protein2']]
    .drop_duplicates()
    .reset_index(drop=True)
)

# 3) Rename columns to match the rest of your pipeline
filtered = filtered.rename(columns={'protein1':'source','protein2':'target'})

# 4) Save it
filtered.to_csv('D:/IUB/Classes/ML for Bioinformatics/Project/data/string_edges_filtered.csv', index=False)
print(f"Saved {len(filtered)} edges to string_edges_filtered.csv")

Saved 841068 edges to string_edges_filtered.csv


In [45]:
#### 1. Imports and device setup
import os
import numpy as np
import pandas as pd
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

# PyTorch Geometric imports
import torch
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool

# For cross-validation & hyperparameter tuning
from sklearn.model_selection import StratifiedKFold

# For interpretability
from torch_geometric.explain import GNNExplainer

In [ ]:
#### 2. Load PPI network and build edge_index
# Assume edges.csv has columns: source, target
ppi = pd.read_csv('D:/IUB/Classes/ML for Bioinformatics/Project/data/string_edges_filtered.csv')  # filtered by cutoff
# Map ENSP IDs to integer indices
genes = sorted(set(ppi.source) | set(ppi.target))
gene2idx = {g: i for i, g in enumerate(genes)}
# Build edge index
edge_index = torch.tensor(
    [[gene2idx[s], gene2idx[t]] for s, t in zip(ppi.source, ppi.target)] +
    [[gene2idx[t], gene2idx[s]] for s, t in zip(ppi.source, ppi.target)],
    dtype=torch.long
).t().contiguous().to(torch_device)
num_nodes = len(genes)


In [ ]:
labels_kim = pd.read_csv('D:/IUB/Classes/ML for Bioinformatics/Project/data/Kim_et_al/response_labels.csv', index_col=0)['response']  # Series of 0/1

In [ ]:
# Align to PPI node order
expr_kim = expr_df.reindex(genes).fillna(0)
X_kim = torch.tensor(expr_kim.values.T, dtype=torch.float)
y_kim = torch.tensor(labels_kim.loc[expr_kim.columns].values, dtype=torch.long)

In [ ]:
#### 4. Build Data objects for PyG
dataset_kim = []
for i in range(X_kim.size(0)):
    data = Data(x=X_kim[i].unsqueeze(1), edge_index=edge_index, y=y_kim[i].view(1))
    dataset_kim.append(data)


In [ ]:
#### 5. Define a simple GCN model
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.5):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin   = torch.nn.Linear(hidden_channels, out_channels)
        self.dropout = dropout

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)
        return self.lin(x)

In [ ]:
#### 6. Training with CV, early stopping, and validation‐loss plotting
import time
import torch
import torch.nn.functional as F
from torch_geometric.loader import DataLoader
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt

# 1) Start overall CV timer
cv_start = time.time()

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
all_metrics = []

for fold, (train_idx, test_idx) in enumerate(skf.split(X_kim, y_kim.numpy()), start=1):
    print(f"\n=== Fold {fold} ===")
    fold_start = time.time()
    val_losses = []

    # 2) Prepare data loaders
    train_loader = DataLoader([dataset_kim[i] for i in train_idx], batch_size=8, shuffle=True)
    test_loader  = DataLoader([dataset_kim[i] for i in test_idx],  batch_size=8)

    # 3) Initialize model & optimizer
    model     = GCN(in_channels=1, hidden_channels=64, out_channels=2).to(torch_device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-4)

    best_val_loss = float('inf')
    patience = 10
    counter = 0
    max_epochs = 100

    # 4) Epoch loop
    for epoch in range(1, max_epochs + 1):
        epoch_start = time.time()
        model.train()
        total_loss = 0.0

        for batch in train_loader:
            batch = batch.to(torch_device)
            optimizer.zero_grad()
            out = model(batch)
            loss = F.cross_entropy(out, batch.y.view(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # 5) Compute & record validation loss
        val_loss = total_loss / len(train_loader)
        val_losses.append(val_loss)
        epoch_time = time.time() - epoch_start
        print(f"Fold {fold}, Epoch {epoch:03d} — val_loss: {val_loss:.4f}, time: {epoch_time:.2f}s")

        # 6) Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), f'best_model_fold{fold}.pt')
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f"Stopping early at epoch {epoch} (no improvement for {patience} epochs).")
                break

    # 7) Plot the validation‐loss curve for this fold
    plt.figure()
    plt.plot(range(1, len(val_losses) + 1), val_losses, marker='o')
    plt.xlabel("Epoch")
    plt.ylabel("Validation Loss")
    plt.title(f"Fold {fold} Validation Loss Curve")
    plt.tight_layout()
    plt.show()

    # 8) Evaluate on the test split
    model.load_state_dict(torch.load(f'best_model_fold{fold}.pt'))
    model.eval()
    correct = total = 0
    for batch in test_loader:
        batch = batch.to(torch_device)
        pred = model(batch).argmax(dim=1)
        correct += int((pred == batch.y.view(-1)).sum())
        total += batch.num_graphs
    acc = correct / total
    all_metrics.append(acc)

    fold_time = time.time() - fold_start
    print(f"Fold {fold} completed in {fold_time:.2f}s — accuracy: {acc:.4f}")

# 9) Overall CV summary
total_cv_time = time.time() - cv_start
print(f"\n5-fold Accuracies: {all_metrics}")
print(f"Total CV training time: {total_cv_time:.2f}s")
